<a href="https://colab.research.google.com/github/ZhongyuZhao/Coursera_Capstone/blob/main/Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook will be mainly used for Coursera capstone project.**

In [1]:
import pandas as pd
import numpy as np
import requests

## Data Extraction

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_url = requests.get(url)
wiki_data = pd.read_html(wiki_url.text)
wiki_data

[    Postal Code  ...                                      Neighbourhood
 0           M1A  ...                                       Not assigned
 1           M2A  ...                                       Not assigned
 2           M3A  ...                                          Parkwoods
 3           M4A  ...                                   Victoria Village
 4           M5A  ...                          Regent Park, Harbourfront
 ..          ...  ...                                                ...
 175         M5Z  ...                                       Not assigned
 176         M6Z  ...                                       Not assigned
 177         M7Z  ...                                       Not assigned
 178         M8Z  ...  Mimico NW, The Queensway West, South of Bloor,...
 179         M9Z  ...                                       Not assigned
 
 [180 rows x 3 columns],
                                                   0   ...   17
 0                               

In [3]:
wiki_data = wiki_data[0]

In [4]:
#we only need first table, so select it as wiki_data and drop 'not assigned' column.
df = wiki_data[~wiki_data['Borough'].isin(['Not assigned'])]
df.groupby(['Postal Code'])
df = df.reset_index()
df.drop(['index'],axis = 'columns', inplace = True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
data = pd.read_csv("https://cocl.us/Geospatial_data")
cols_to_use = data.columns.difference(df.columns)
df_combine = df.join(data.set_index('Postal Code'), on='Postal Code', how='inner')
df_combine

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [6]:
df_ny = df_combine[df['Borough'].isin(['North York'])]
df_ny

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M3B,North York,Don Mills,43.745906,-79.352188
10,M6B,North York,Glencairn,43.709577,-79.445073
13,M3C,North York,Don Mills,43.725900,-79.340923
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
34,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


In [7]:
df_dt = df_combine[df['Borough'].isin(['Downtown Toronto'])]
df_dt

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


## Mapping

In [8]:
from geopy.geocoders import Nominatim

In [9]:
#get latitude and longitude of North York
address = 'North York, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [10]:
# create map of New York using latitude and longitude values
import folium
map_toronto_ny = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_ny['Latitude'], df_ny['Longitude'], df_ny['Borough'], df_ny['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_ny)  
    
map_toronto_ny

In [11]:
map_toronto_dt = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighbourhood in zip(df_dt['Latitude'], df_dt['Longitude'], df_dt['Borough'], df_dt['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_dt)  
    
map_toronto_dt

## Nearby Venues

In [12]:
CLIENT_ID = 'AUX0SNLBUOS4JO5H3YFS1VQ44HSUH3KWG5GPY1TNNES2LQDO' # your Foursquare ID
CLIENT_SECRET = 'MEKYKTHN1O2YVYICDD2D3T0T0LHT20A0NK3CDBC3XUDS2DKG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AUX0SNLBUOS4JO5H3YFS1VQ44HSUH3KWG5GPY1TNNES2LQDO
CLIENT_SECRET:MEKYKTHN1O2YVYICDD2D3T0T0LHT20A0NK3CDBC3XUDS2DKG


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
NY_venues= getNearbyVenues(df_ny['Neighbourhood'], df_ny['Latitude'], df_ny['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [15]:
DT_venues= getNearbyVenues(df_dt['Neighbourhood'], df_dt['Latitude'], df_dt['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


## Onehot Encoding

In [18]:
#onehot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 
NY_onehot['Neighborhood'] = NY_venues['Neighborhood']

# move neighborhood column to the first column
dt_fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
ny_fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])

DT_onehot = DT_onehot[dt_fixed_columns]
NY_onehot = NY_onehot[ny_fixed_columns]

In [24]:
DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index()
DT_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.018182,0.000000,0.0,0.0,0.0000,0.0,0.0000,0.018182,0.036364,0.000000,0.000000,0.0,0.018182,0.018182,0.0,0.036364,0.0,0.0,0.018182,0.000000,0.000000,0.000000,0.0,0.018182,0.0,0.000000,0.0,0.000000,...,0.000000,0.036364,0.0,0.000000,0.000000,0.000000,0.018182,0.0,0.000000,0.036364,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.018182,0.0,0.018182,0.000000,0.0,0.018182,0.0,0.018182,0.0,0.0,0.000000,0.018182,0.000000,0.000000,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0000,0.000000,0.000000,0.000000,0.058824,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.058824,0.000000,0.058824,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.058824,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.058824,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,Central Bay Street,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.016129,0.0,0.0,0.0000,0.0,0.0000,0.000000,0.000000,0.016129,0.016129,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.032258,0.0,0.032258,...,0.000000,0.016129,0.0,0.000000,0.032258,0.000000,0.048387,0.0,0.000000,0.016129,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.016129,0.0,0.000000,0.0,0.016129,0.000000,0.0,0.016129,0.0,0.000000,0.0,0.0,0.016129,0.016129,0.000000,0.000000,0.0,0.0,0.016129,0.0,0.0,0.016129,0.0
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0625,0.0,0.0625,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.062500,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Church and Wellesley,0.025316,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.012658,0.0,0.0,0.000000,0.000000,0.012658,0.000000,0.0,0.012658,0.0,0.012658,0.0,0.025316,...,0.000000,0.037975,0.0,0.012658,0.000000,0.012658,0.000000,0.0,0.012658,0.000000,0.0,0.0,0.0,0.012658,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.012658,0.012658,0.0,0.063291,0.0,0.000000,0.0,0.0,0.000000,0.012658,0.012658,0.012658,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [25]:
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()
NY_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,...,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Martial Arts School,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.095238,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.047619,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.047619,0.0,0.000000,0.095238,0.00,0.0,0.0,0.0,0.047619,0.0,0.000000,0.047619,0.000000,0.0,...,0.047619,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.047619,0.0,0.000000,0.0,0.047619,0.047619,0.0,0.0,0.0,0.00,0.0,0.047619,0.0,0.047619,0.047619,0.000000,0.000000,0.0,0.047619,0.047619,0.0,0.00,0.0,0.00,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.250000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.250000,0.0,0.000000,0.250000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.250000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.04,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.04,0.000000,0.0,0.0,0.0,0.0,0.000000,0.04,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.080000,0.04,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.080000,0.000000,0.0,0.04,0.040000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.040000,0.040000,0.0,0.0,0.0,0.04,0.0,0.040000,0.0,0.080000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.0,0.08,0.0,0.04,0.0,0.0,0.0
3,Don Mills,0.0,0.000000,0.00,0.037037,0.0,0.037037,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.074074,0.037037,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.037037,0.0,0.037037,0.037037,0.0,0.074074,0.074074,0.00,0.0,0.0,0.0,0.000000,0.0,0.037037,0.000000,0.037037,0.0,...,0.000000,0.037037,0.074074,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.074074,0.0,0.037037,0.037037,0.000000,0.037037,0.0,0.037037,0.037037,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,Downsview,0.0,0.071429,0.00,0.000000,0.0,0.000000,0.071429,0.0,0.071429,0.0,0.071429,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.071429,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.00,0.071429,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.142857,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.071429,0.071429,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0


In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
#Print each neighborhood along with top 5 venues in a pandas dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_NY = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_DT = pd.DataFrame(columns=columns)

neighborhoods_venues_sorted_NY['Neighborhood'] = NY_grouped['Neighborhood']
neighborhoods_venues_sorted_DT['Neighborhood'] = DT_grouped['Neighborhood']


for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted_NY.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)
for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted_DT.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

In [30]:
neighborhoods_venues_sorted_DT.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Restaurant,Bakery,Beer Bar,Hotel,Eastern European Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Plane,Rental Car Location,Coffee Shop,Bar,Harbor / Marina
2,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Comic Shop,Restaurant,Portuguese Restaurant
3,Christie,Grocery Store,Café,Park,Nightclub,Coffee Shop,Restaurant,Italian Restaurant,Candy Store,Athletics & Sports,Baby Store
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Fast Food Restaurant


## Clustering

In [32]:
from sklearn.cluster import KMeans

In [33]:
kclusters = 5

NY_grouped_clustering = NY_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 3], dtype=int32)

In [34]:
neighborhoods_venues_sorted_NY.insert(0, 'Cluster Labels', kmeans.labels_)

In [36]:
NY_merged = df_ny

# merge two datasets to add latitude/longitude for each neighborhood
NY_merged = NY_merged.join(neighborhoods_venues_sorted_NY.set_index('Neighborhood'), on='Neighbourhood')

NY_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Hotel,Department Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,French Restaurant,Coffee Shop,Hockey Arena,Portuguese Restaurant,Vietnamese Restaurant,Department Store,Chocolate Shop,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Accessories Store,Supermarket,Café
7,M3B,North York,Don Mills,43.745906,-79.352188,0,Gym,Clothing Store,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Caribbean Restaurant,Café,Italian Restaurant
10,M6B,North York,Glencairn,43.709577,-79.445073,0,Park,Pizza Place,Bakery,Pub,Japanese Restaurant,Vietnamese Restaurant,Deli / Bodega,Chocolate Shop,Clothing Store,Coffee Shop


In [37]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [39]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], NY_merged['Neighbourhood'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now we add the old address to 'North York' set and do clustering again, see what happens.


In [60]:
insert_row = DT_venues[DT_venues['Neighborhood'].str.contains('Provincial')]
NY_venues_plus = NY_venues.copy()
NY_venues_plus = NY_venues_plus.append(insert_row)
NY_venues_plus

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Towns On The Ravine,43.754754,-79.332552,Hotel
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
...,...,...,...,...,...,...,...
72,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.658906,-79.388696,Coffee Shop
73,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.659415,-79.391221,Coffee Shop
74,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Gallery Grill,43.663841,-79.394309,Restaurant
75,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Arbor Room,43.663604,-79.394615,College Cafeteria


In [63]:
NY_venus_plus = NY_venues_plus.reset_index()
NY_venus_plus

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,1,Parkwoods,43.753259,-79.329656,Towns On The Ravine,43.754754,-79.332552,Hotel
2,2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
...,...,...,...,...,...,...,...,...
265,72,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.658906,-79.388696,Coffee Shop
266,73,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.659415,-79.391221,Coffee Shop
267,74,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Gallery Grill,43.663841,-79.394309,Restaurant
268,75,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Arbor Room,43.663604,-79.394615,College Cafeteria


Do onehot encoding again